In [190]:
import sys

# 打印 Python 解释器的路径
print("Python executable:", sys.executable)

# 打印 Python 版本
print("Python version:", sys.version)

Python executable: /data/mr423/miniconda3-2d/envs/scgpt/bin/python
Python version: 3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]


In [191]:
import tempfile
import os

# new_temp_dir = "/data/mr423/tmp"
# os.makedirs(new_temp_dir, exist_ok=True)
# os.environ["TMPDIR"] = new_temp_dir  # 对于 Unix/Linux
# os.environ["TMP"] = new_temp_dir  # 适用于所有平台

# 检查 tempfile 模块使用的临时文件目录
temp_dir = tempfile.gettempdir()
print("Updated temp directory:", temp_dir)

Updated temp directory: /data/mr423/tmp


In [192]:
import sys

new_path = '/data/mr423/scGPT/'

if new_path not in sys.path:
    sys.path.insert(0, new_path)

import importlib
import scgpt
importlib.reload(scgpt)

<module 'scgpt' from '/data/mr423/scGPT/scgpt/__init__.py'>

In [193]:
print(scgpt.__file__)

/data/mr423/scGPT/scgpt/__init__.py


In [194]:
# %%
import copy
import gc
import json
import os
from pathlib import Path
import shutil
import sys
import time
import traceback
from typing import List, Tuple, Dict, Union, Optional
import warnings
import pandas as pd
# from . import asyn
import pickle
import torch
from anndata import AnnData
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import wandb
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from sklearn.metrics import confusion_matrix

# sys.path.insert(0, "../")


import scgpt as scg
from scgpt.model import TransformerModel, AdversarialDiscriminator
from scgpt.tokenizer import tokenize_and_pad_batch, random_mask_value
from scgpt.loss import (
    masked_mse_loss,
    masked_relative_error,
    criterion_neg_log_bernoulli,
)
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.preprocess import Preprocessor
from scgpt import SubsetsBatchSampler
from scgpt.utils import set_seed, category_str2int, eval_scib_metrics

sc.set_figure_params(figsize=(6, 6))
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

In [195]:
hyperparameter_defaults = dict(
    seed=0,
    dataset_name="protein",
    do_train=True,
    load_model="../pre_trained_model/scGPT_blood",
    mask_ratio=0.0,
    epochs=30,
    n_bins=101,
    MVC=False, # Masked value prediction for cell embedding
    ecs_thres=0.0, # Elastic cell similarity objective, 0.0 to 1.0, 0.0 to disable
    dab_weight=0.0,
    lr=1e-3,
    batch_size=32,
    layer_size=128,
    nlayers=4,  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    nhead=4,  # number of heads in nn.MultiheadAttention
    dropout=0.5,  # dropout probability
    schedule_ratio=0.9,  # ratio of epochs for learning rate schedule
    save_eval_interval=5,
    fast_transformer=True,
    pre_norm=False,
    amp=True,  # Automatic Mixed Precision
    include_zero_gene = True,
    freeze = True, #freeze
    DSBN = False,  # Domain-spec batchnorm
)

In [196]:
run = wandb.init(
    config=hyperparameter_defaults,
    project="scGPT",
    reinit=True,
    name="ann1",
    settings=wandb.Settings(start_method="fork"),
)
config = wandb.config
print(config)

set_seed(config.seed)

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,▅▇▆▅▆▂▃▁▁▅▃▇▅▁▂▅▃█▆▄▃▆▅▅▃▅▇█▄▅▅▄▃▄▅▆▆▇▇▅
info/post_freeze_param_count,▁
info/pre_freeze_param_count,▁
train_loss,█▅▂▄▂▄▄▃▅▄▄▂▂▄▃▃▃▂▁▂▄▂▃▂▃▃▂▃▂▂▂▄▄▃▃▂▄▂▂▃
epoch,1
error_rate,0.1875
info/post_freeze_param_count,19729417
info/pre_freeze_param_count,38510089
train_loss,1.95334


{'seed': 0, 'dataset_name': 'protein', 'do_train': True, 'load_model': '../pre_trained_model/scGPT_blood', 'mask_ratio': 0.0, 'epochs': 30, 'n_bins': 101, 'MVC': False, 'ecs_thres': 0.0, 'dab_weight': 0.0, 'lr': 0.001, 'batch_size': 32, 'layer_size': 128, 'nlayers': 4, 'nhead': 4, 'dropout': 0.5, 'schedule_ratio': 0.9, 'save_eval_interval': 5, 'fast_transformer': True, 'pre_norm': False, 'amp': True, 'include_zero_gene': True, 'freeze': True, 'DSBN': False}


In [239]:
# settings for input and preprocessing
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
mask_ratio = config.mask_ratio
mask_value = "auto"  # for masked values, now it should always be auto

include_zero_gene = config.include_zero_gene  # if True, include zero genes among hvgs in the training
max_seq_len = 3001
n_bins = config.n_bins

# input/output representation
'''
input_style 决定了模型输入数据的预处理和表示方式。常见的选项包括：
"normed_raw"：归一化的原始数据
"log1p"：对数据进行 log(1 + x) 转换
"binned"：将数据分箱处理
'''
input_style = "binned"  # "normed_raw", "log1p", or "binned"
output_style = "binned"  # "normed_raw", "log1p", or "binned"

# settings for training

'''
MLM (Masked Language Modeling) 是一种预训练目标，广泛用于像 BERT 这样的 transformer 模型中。
它通过随机遮蔽输入文本中的某些 token，然后要求模型预测这些被遮蔽的 token，从而学习上下文信息

CLS:(Celltype Classification Objective) 是指模型在训练过程中包含一个细胞类型分类的目标。这通常应用于生物信息学领域，尤其是在单细胞 RNA 测序数据分析中。

ADV (Adversarial Training for Batch Correction) 是一种训练策略，用于纠正批次效应。
批次效应是指在不同实验批次中数据分布的变化，这在生物信息学数据处理中是一个常见问题。

CCE (Contrastive Cell Embedding Objective) 是一种训练目标，通过对比学习（contrastive learning）来优化细胞嵌入表示。
如果 config.ecs_thres 大于 0，则启用此目标。
对比学习通过将相似的样本拉近、不同的样本推远来学习数据的结构。

ECS (Elastic Cell Similarity) 是一种训练目标，旨在优化细胞嵌入的相似性。通过设置一个阈值，模型会尝试使相似的细胞在嵌入空间中更接近。
如果 config.ecs_thres 大于 0，则启用此目标。

DAB (Domain Adaptation by Reverse Backpropagation) 是一种对抗训练策略，用于跨域适应。通过反向传播调整模型参数，使模型能够在不同域间更好地泛化。
设置为 False，表示不使用此策略。如果设置为 2，则使用单独的优化器进行训练。

INPUT_BATCH_LABELS: 控制是否使用输入的批次标签。这些标签可以帮助 MLM 和 MVC 目标，但不用于分类器

-----------------------------------------------------------------------------------

input_emb_style 参数决定了输入数据在模型中的嵌入方式。根据不同的数据类型和任务需求，选择合适的嵌入方式可以显著影响模型的性能。

Category ("category")
解释：选择 "category" 作为嵌入风格时，输入数据被认为是类别型数据。这种方式用于离散的类别数据，如分类标签、离散事件等。
用途：适用于分类特征，例如细胞类型标签、基因突变状态等。
实现：输入数据通常会进行独热编码（one-hot encoding）或嵌入编码（embedding encoding）处理

Continuous ("continuous")
解释：选择 "continuous" 作为嵌入风格时，输入数据被认为是连续型数据。这种方式通常用于数值数据或有序数据。
用途：适用于数值型特征，例如基因表达数据、价格、温度等连续型变量。
实现：输入数据直接以其原始数值形式输入模型，可能会进行标准化或归一化处理以提高模型性能。

Scaling ("scaling")
解释：选择 "scaling" 作为嵌入风格时，输入数据在输入模型之前会进行缩放处理。这种方式可以用于任意类型的数据，通过缩放使数据在统一的尺度范围内，通常在 [0, 1] 之间。
用途：适用于需要统一尺度的特征，以防止某些特征在训练过程中由于数值范围过大或过小而影响模型性能。
实现：输入数据通常会进行最小-最大缩放（min-max scaling）处理。

-----------------------------------------------------------------------------------

cell_emb_style: 定义如何从模型的输出中提取单个句子或文档的嵌入表示（embedding representation）

Average Pooling ("avg-pool") 平均池化是对所有 token 的嵌入取平均值，以获取整个序列的表示。
Weighted Pooling ("w-pool") 加权池化是对所有 token 的嵌入进行加权平均
CLS Token Embedding ("cls") 用于分类任务，因为 [CLS] token 的嵌入专门设计用于捕获全局信息。

'''

MLM = False  # whether to use masked language modeling, currently it is always on.
CLS = True  # celltype classification objective
ADV = False  # Adversarial training for batch correction
CCE = False  # Contrastive cell embedding objective
MVC = config.MVC  # Masked value prediction for cell embedding
ECS = config.ecs_thres > 0  # Elastic cell similarity objective
DAB = False


INPUT_BATCH_LABELS = False  # TODO: have these help MLM and MVC, while not to classifier

input_emb_style = "category"  # "category" or "continuous" or "scaling"

cell_emb_style = "cls"  # "avg-pool" or "w-pool" or "cls"


# adv_E_delay_epochs：编码器的对抗训练延迟几个 epoch 开始。
# adv_D_delay_epochs：分类器的对抗训练延迟几个 epoch 开始。
adv_E_delay_epochs = 0  # delay adversarial training on encoder for a few epochs
adv_D_delay_epochs = 0


mvc_decoder_style = "inner product"  # MVC 目标的解码器风格
ecs_threshold = config.ecs_thres  # 控制 ECS 目标的相似性阈值
dab_weight = config.dab_weight  # 控制 DAB 目标的权重,调整对抗训练在整体训练过程中的影响力。

explicit_zero_prob = MLM and include_zero_gene  # whether explicit bernoulli for zeros
print("explicit_zero_prob: ", explicit_zero_prob)

do_sample_in_train = False and explicit_zero_prob  # sample the bernoulli in training

# 决定是否在每个序列批次进行采样
per_seq_batch_sample = False

# settings for optimizer
lr = config.lr  # TODO: test learning rate ratio between two tasks
lr_ADV = 1e-3  # learning rate for discriminator, used when ADV is True

batch_size = config.batch_size
eval_batch_size = config.batch_size
epochs = config.epochs
schedule_interval = 1

# settings for the model
fast_transformer = config.fast_transformer
fast_transformer_backend = "flash"  # "linear" or "flash"  选择不同的 Transformer 编码器（Encoder）实现方式: linear attention or flash attention

embsize = config.layer_size  # embedding dimension
d_hid = config.layer_size  # dimension of the feedforward network in TransformerEncoder
nlayers = config.nlayers  # number of TransformerEncoderLayer in TransformerEncoder
nhead = config.nhead  # number of heads in nn.MultiheadAttention
dropout = config.dropout  # dropout probability

# logging
log_interval = 60  # iterations
save_eval_interval = config.save_eval_interval  # epochs
do_eval_scib_metrics = True

explicit_zero_prob:  False


In [198]:
# %% validate settings
assert input_style in ["normed_raw", "log1p", "binned"]
assert output_style in ["normed_raw", "log1p", "binned"]
assert input_emb_style in ["category", "continuous", "scaling"]

if input_style == "binned":
    if input_emb_style == "scaling":
        raise ValueError("input_emb_style `scaling` is not supported for binned input.")
elif input_style == "log1p" or input_style == "normed_raw":
    if input_emb_style == "category":
        raise ValueError(
            "input_emb_style `category` is not supported for log1p or normed_raw input."
        )

if input_emb_style == "category":
    mask_value = n_bins + 1
    pad_value = n_bins  # for padding gene expr values
    n_input_bins = n_bins + 2
else:
    mask_value = -1
    pad_value = -2
    n_input_bins = n_bins

if ADV and DAB:
    raise ValueError("ADV and DAB cannot be both True.")
DAB_separate_optim = True if DAB > 1 else False

In [199]:
dataset_name = config.dataset_name
save_dir = Path(f"./save/dev_{dataset_name}-{time.strftime('%b%d-%H-%M')}/")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"save to {save_dir}")
logger = scg.logger
scg.utils.add_file_handler(logger, save_dir / "run.log")

save to save/dev_protein-Jul25-12-05


In [200]:
adata = sc.read("./data/process_OLINK_data_nanToZero.h5ad")
# adata_test = sc.read("./data/protein_adata_withsex_notContainNaN_test.h5ad")

In [201]:
print(adata.shape)
# print(adata_test.shape)

(6000, 2920)


In [202]:
# sc.pp.neighbors(adata)
# sc.tl.umap(adata)

# sc.pp.neighbors(adata_test)
# sc.tl.umap(adata_test)

In [203]:
adata.obs["individual"] = adata.obs.index
# adata_test.obs["individual"] = adata_test.obs.index

In [204]:
# 将细胞类型信息转换为分类类型
# 将细胞类型信息从字符串转换为分类数据类型 (category)
# adata.obs["sextype"] = adata.obs["sex"].astype("category")
# adata_test.obs["sextype"] = adata_test.obs["sex"].astype("category")

# 设置批次信息，为了合并数据集后可以区分训练和测试数据集
# adata.obs["batch_id"]  = adata.obs["str_batch"] = "0"
# adata_test.obs["batch_id"]  = adata_test.obs["str_batch"] = "1"      

# 设置基因名称为变量索引
adata.var.set_index(adata.var["gene_name"], inplace=True)
# adata_test.var.set_index(adata.var["gene_name"], inplace=True)

data_is_raw = False
filter_gene_by_counts = False

# adata_test_raw = adata_test.copy() # 备份测试数据集

# adata = adata.concatenate(adata_test, batch_key="str_batch") # 合并训练和测试数据集，使用 `str_batch` 作为批次键

In [205]:
gene_names = adata.var.index.tolist()
len(gene_names)

2920

In [206]:
# # 合并后

# # make the batch category column
# # 将批次标签转换为类别编码，并存储在 batch_id 列中
# batch_id_labels = adata.obs["str_batch"].astype("category").cat.codes.values
# adata.obs["batch_id"] = batch_id_labels

# # 将细胞类型标签转换为类别编码，并存储在 celltype_id 列中
# sextype_id_labels = adata.obs["sex"].astype("category").cat.codes.values
# num_types = len(np.unique(sextype_id_labels))
# sextypes = adata.obs["sex"].unique()

# adata.obs["sex_id"] = sextype_id_labels # 将细胞类型 ID 添加到观察数据中

# # 计算细胞类型的数量，并生成 ID 到类型的映射字典
# id2type = dict(enumerate(adata.obs["sex"].astype("category").cat.categories))
# print(len(id2type))


#  # 更新基因名称为变量列
# adata.var["gene_name"] = adata.var.index.tolist()

In [207]:
age_group_counts = adata.obs['Age_Group'].value_counts().sort_index()
age_group_counts

Age_Group
50-55       8
55-60     424
60-65     726
65-70     839
70-75    1023
75-80    1463
80-85    1253
85-90     264
Name: count, dtype: int64

In [208]:
adata.obs['Age_Group_Code'] = adata.obs['Age_Group'].cat.codes

In [209]:
age_group_code_counts = adata.obs['Age_Group_Code'].value_counts()
age_group_code_counts

Age_Group_Code
5    1463
6    1253
4    1023
3     839
2     726
1     424
7     264
0       8
Name: count, dtype: int64

In [210]:
adata.obs

,sex,DoB_year,Age,Age_Group,individual,Age_Group_Code
Id,,,,,,
1000221,0,1945.0,79,75-80,1000221,5
1000326,1,1942.0,82,80-85,1000326,6
1000662,0,1956.0,68,65-70,1000662,3
1000823,0,1964.0,60,60-65,1000823,2
1000945,1,1948.0,76,75-80,1000945,5
...,...,...,...,...,...,...
1733048,1,1946.0,78,75-80,1733048,5
1733304,0,1945.0,79,75-80,1733304,5
1733421,1,1962.0,62,60-65,1733421,2


In [211]:
# agetype_id_labels = adata.obs["Age_Group"].cat.codes.values
# num_types = len(np.unique(agetype_id_labels))
# print(num_types)

# agetype = adata.obs["Age_Group"].unique()

# adata.obs["agetype_id"] = agetype_id_labels # 将细胞类型 ID 添加到观察数据中

# # 计算细胞类型的数量，并生成 ID 到类型的映射字典
# id2type = dict(enumerate(adata.obs["Age_Group"].astype("category").cat.categories))
# print(len(id2type))


 # 更新基因名称为变量列
adata.var["gene_name"] = adata.var.index.tolist()

In [162]:
unique_clusters = adata.obs['Age_Group_Code'].unique()

print(unique_clusters)
num_clusters = len(unique_clusters)
print(f"Total {num_clusters} of /the cluster labels.")

cluster_sizes = adata.obs['Age_Group_Code'].value_counts()
print(cluster_sizes)

[5 6 3 2 7 4 1 0]
Total 8 of /the cluster labels.
Age_Group_Code
5    1463
6    1253
4    1023
3     839
2     726
1     424
7     264
0       8
Name: count, dtype: int64


In [163]:
# 检查adata.X中的负值
if np.any(adata.X < 0):
    print("数据中包含负值")
else:
    print("数据中不包含负值")

数据中不包含负值


In [164]:
adata.X

array([[0.75756712, 1.05973776, 0.81593332, ..., 1.15026781, 0.        ,
        0.        ],
       [1.06278189, 2.29367793, 0.        , ..., 0.98589464, 0.73487288,
        0.        ],
       [1.04225113, 1.41731008, 0.94009768, ..., 0.96449787, 0.96967555,
        1.18637255],
       ...,
       [0.92070645, 1.19198703, 1.19013236, ..., 1.19356213, 0.52167633,
        0.8199579 ],
       [0.87312188, 0.59920075, 0.78309161, ..., 0.84485812, 2.41546227,
        1.1980134 ],
       [1.08649695, 1.07747466, 0.96337697, ..., 0.90819564, 3.28254599,
        0.86678221]])

### Match the genes od the pre-train model¶

In [165]:
if config.load_model is not None:
    model_dir = config.load_model
    model_config_file = model_dir + "/args.json"
    model_file = model_dir + "/best_model.pt"
    vocab_file = model_dir + "/vocab.json"

    vocab = GeneVocab.from_file(vocab_file)
    shutil.copy(vocab_file, save_dir / "vocab.json")
    for s in special_tokens:
        if s not in vocab:
            vocab.append_token(s)

    '''
    标记和筛选在词汇表中的基因
    # 在 adata.var 中添加一列 id_in_vocab，标记基因是否在词汇表中。如果基因在词汇表中，则值为 1，否则为 -1
    '''
    
    # adata
    adata.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata.var["gene_name"]
    ]
    gene_ids_in_vocab = np.array(adata.var["id_in_vocab"])
    logger.info(
        f"adata match {np.sum(gene_ids_in_vocab >= 0)}/{len(gene_ids_in_vocab)} genes "
        f"in vocabulary of size {len(vocab)}."
    )
    adata = adata[:, adata.var["id_in_vocab"] >= 0]

    # print(adata.shape)
    gene_in_adata = adata.var["gene_name"].tolist()


    # 加载模型配置并覆盖默认配置
    with open(model_config_file, "r") as f:
        model_configs = json.load(f)
    logger.info(
        f"Resume model from {model_file}, the model args will override the "
        f"config {model_config_file}."
    )
    embsize = model_configs["embsize"]
    nhead = model_configs["nheads"]
    d_hid = model_configs["d_hid"]
    nlayers = model_configs["nlayers"]
    n_layers_cls = model_configs["n_layers_cls"]

    print("embsize:" ,embsize)
    print("nhead:" ,nhead)
    print("d_hid:" ,d_hid)
    print("nlayers:" ,nlayers)
    print("n_layers_cls:" ,n_layers_cls)

scGPT - INFO - adata match 2889/2920 genes in vocabulary of size 36574.
scGPT - INFO - Resume model from ../pre_trained_model/scGPT_blood/best_model.pt, the model args will override the config ../pre_trained_model/scGPT_blood/args.json.
embsize: 512
nhead: 8
d_hid: 512
nlayers: 12
n_layers_cls: 3


In [166]:
print(adata)
# print(adata_test)

View of AnnData object with n_obs × n_vars = 6000 × 2889
    obs: 'sex', 'DoB_year', 'Age', 'Age_Group', 'individual', 'Age_Group_Code'
    var: 'gene_name', 'id_in_vocab'


In [169]:
# set up the preprocessor, use the args to config the workflow
# 应用了数据预处理器，用于对 AnnData 对象进行一系列的预处理操作。这些操作包括基因过滤、细胞过滤、数据归一化、对数转换和数据分箱

preprocessor = Preprocessor(
    use_key="X",  # 在 adata.layers 中使用的原始数据的键
    filter_gene_by_counts=filter_gene_by_counts,  # 步骤1: 是否通过基因计数过滤
    filter_cell_by_counts=False,  # 步骤2: 是否通过细胞计数过滤
    
    normalize_total=1,  # 步骤3: 是否归一化原始数据以及归一化的总和  一行相加的和
    result_normed_key="X_normed",  # 在 adata.layers 中存储归一化数据的键
    
    log1p=False,  # 步骤4: 是否对归一化数据进行 log1p 转换
    result_log1p_key="X_log1p",  # 存储 log1p 转换数据的键
    
    subset_hvg=False,  # 步骤5: 是否筛选高变基因
    hvg_flavor="seurat_v3" if data_is_raw else "cell_ranger",  # 高变基因筛选的风格
    
    binning=n_bins,  # 步骤6: 是否对原始数据进行分箱处理以及分箱数量
    result_binned_key="X_binned",  # 在 adata.layers 中存储分箱数据的键 !!!
)

# 分割训练集和测试集，通过str_batch
# adata_test = adata[adata.obs["str_batch"] == "1"]
# adata = adata[adata.obs["str_batch"] == "0"]


# 应用预处理器
preprocessor(adata, batch_key=None)
# preprocessor(adata_test, batch_key=None)

scGPT - INFO - Normalizing total counts ...
scGPT - INFO - 111 ...
scGPT - INFO - Binning data ...


In [170]:
print(adata)
# print(adata_test)

AnnData object with n_obs × n_vars = 6000 × 2889
    obs: 'sex', 'DoB_year', 'Age', 'Age_Group', 'individual', 'Age_Group_Code'
    var: 'gene_name', 'id_in_vocab'
    obsm: 'bin_edges'
    layers: 'X_normed', 'X_binned'


In [171]:
adata.X

array([[0.75756712, 1.05973776, 0.81593332, ..., 1.15026781, 0.        ,
        0.        ],
       [1.06278189, 2.29367793, 0.        , ..., 0.98589464, 0.73487288,
        0.        ],
       [1.04225113, 1.41731008, 0.94009768, ..., 0.96449787, 0.96967555,
        1.18637255],
       ...,
       [0.92070645, 1.19198703, 1.19013236, ..., 1.19356213, 0.52167633,
        0.8199579 ],
       [0.87312188, 0.59920075, 0.78309161, ..., 0.84485812, 2.41546227,
        1.1980134 ],
       [1.08649695, 1.07747466, 0.96337697, ..., 0.90819564, 3.28254599,
        0.86678221]])

In [172]:
adata.layers['X_normed']

array([[0.78896957, 1.1036657 , 0.84975515, ..., 1.19794837, 0.        ,
        0.        ],
       [0.96572812, 2.08421813, 0.        , ..., 0.89586225, 0.66776392,
        0.        ],
       [0.90196284, 1.2265384 , 0.81355938, ..., 0.83467527, 0.83915603,
        1.02668535],
       ...,
       [0.91068314, 1.1790104 , 1.17717593, ..., 1.18056836, 0.51599707,
        0.81103139],
       [0.95236234, 0.65358142, 0.85416135, ..., 0.9215335 , 2.63467836,
        1.30673951],
       [1.12605316, 1.11670239, 0.99845074, ..., 0.94126041, 3.40205398,
        0.89833924]])

In [173]:
adata.layers['X_binned']

array([[12, 77, 20, ..., 88,  0,  0],
       [51, 96,  0, ..., 34,  6,  0],
       [35, 85, 17, ..., 22, 22, 60],
       ...,
       [28, 72, 72, ..., 72,  2, 14],
       [36,  5, 20, ..., 30, 99, 88],
       [69, 67, 42, ..., 31, 99, 23]])

### 预处理后的数据按照一定比例分割为训练集和验证集¶

In [174]:
adata.obs.index = adata.obs["individual"]
# adata_test.obs.index = adata_test.obs["individual"]

In [176]:
# the values of this map coorespond to the keys in preprocessing
# 通过提供的 input_style，从字典中获取相应的键
input_layer_key = {  
    "normed_raw": "X_normed",
    "log1p": "X_normed",  # log(1+x)
    "binned": "X_binned",
}[input_style]

# layers 是一个字典，可以存储多个数据矩阵，这些矩阵通常是在不同处理步骤中生成的。
# 例如可以在预处理数据时生成归一化数据、对数转换数据或分箱数据，并将它们存储在 layers 中
# 通过 adata.layers[input_layer_key]，可以访问 layers 字典中键为 input_layer_key 的数据层

all_counts = (
    adata.layers[input_layer_key].A
    if issparse(adata.layers[input_layer_key])
    else adata.layers[input_layer_key]
)


# all_counts = X_binned
print(np.allclose(adata.layers['X_binned'], all_counts, atol=1e-9))
print(all_counts.shape)

genes = adata.var["gene_name"].tolist()

# 提取细胞类型标签
cellypes_labels = adata.obs["Age_Group_Code"].tolist()  # make sure count from 0
cellypes_labels = np.array(cellypes_labels)

# # 提取批次标签
# batch_ids = adata.obs["batch_id"].tolist()
# num_batch_types = len(set(batch_ids))
# print("num_batch_types: ",num_batch_types)
# batch_ids = np.array(batch_ids)

# 分割数据集
# 将all_counts、celltypes_labels 和 batch_ids 以相同的比例分为训练集和验证集
(
    train_data,
    valid_data,
    
    train_celltype_labels,
    valid_celltype_labels,
    
    # train_batch_labels,
    # valid_batch_labels,
) = train_test_split(
    all_counts,          # 输入数据
    
    cellypes_labels,    # 细胞类型标签
    
    # batch_ids,           # 批次标签
    
    test_size=0.1,       # 验证集大小设置为 10%
    shuffle=True         # 在划分前随机打乱数据
)

True
(6000, 2889)


In [177]:
print(train_data)
print(train_data.shape)

[[98  6 55 ... 79 99 39]
 [15  3 75 ... 62 54 82]
 [33 73  0 ... 63 58 78]
 ...
 [59 11 53 ... 72 20 96]
 [13  2 51 ... 52 71 87]
 [25 97  2 ... 43 75 39]]
(5400, 2889)


In [213]:
if config.load_model is None:
    vocab = Vocab(
        VocabPybind(genes + special_tokens, None)
    )  # bidirectional lookup [gene <-> int]

vocab.set_default_index(vocab["<pad>"])

gene_ids = np.array(vocab(genes), dtype=int)

print(gene_ids.max())
print(len(gene_ids))

36514
2889


In [214]:
print(vocab["<pad>"])
print(vocab["<cls>"])
print(vocab["<eoc>"])

36571
36572
36573


### 对训练集进行标记化和填充

In [215]:
'''
1. 将train_data 中的基因名称转换为标记
2. 将标记化后的序列填充到固定长度 max_seq_len，不足的部分使用 pad_token 和 pad_value 填充
3. 在每个序列的开头添加 <cls> 标记
'''
# pdb.set_trace()
tokenized_train = tokenize_and_pad_batch(
    train_data,
    gene_ids,
    max_len=max_seq_len, # 填充后的序列长度不会超过原始数据中实际存在的最大长度，即取min(max_seq_len, max_actual_seq_len)
    vocab=vocab,
    pad_token=pad_token,
    pad_value=pad_value,
    append_cls=True,  # append <cls> token at the beginning
    include_zero_gene=include_zero_gene,
)
    
# 对验证集进行标记化和填充
tokenized_valid = tokenize_and_pad_batch(
    valid_data,
    gene_ids,
    max_len=max_seq_len,
    vocab=vocab,
    pad_token=pad_token,
    pad_value=pad_value,
    append_cls=True,
    include_zero_gene=include_zero_gene,
)
logger.info(
    f"train set number of samples: {tokenized_train['genes'].shape[0]}, "
    f"\n\t feature length: {tokenized_train['genes'].shape[1]}"
)
logger.info(
    f"valid set number of samples: {tokenized_valid['genes'].shape[0]}, "
    f"\n\t feature length: {tokenized_valid['genes'].shape[1]}"
)

scGPT - INFO - train set number of samples: 5400, 
	 feature length: 2890
scGPT - INFO - valid set number of samples: 600, 
	 feature length: 2890


In [216]:
print(tokenized_train['genes'])
print(tokenized_train['values'])

tensor([[36572,  8381,  8385,  ...,  4939, 12884, 35421],
        [36572,  8381,  8385,  ...,  4939, 12884, 35421],
        [36572,  8381,  8385,  ...,  4939, 12884, 35421],
        ...,
        [36572,  8381,  8385,  ...,  4939, 12884, 35421],
        [36572,  8381,  8385,  ...,  4939, 12884, 35421],
        [36572,  8381,  8385,  ...,  4939, 12884, 35421]])
tensor([[ 0., 98.,  6.,  ..., 79., 99., 39.],
        [ 0., 15.,  3.,  ..., 62., 54., 82.],
        [ 0., 33., 73.,  ..., 63., 58., 78.],
        ...,
        [ 0., 59., 11.,  ..., 72., 20., 96.],
        [ 0., 13.,  2.,  ..., 52., 71., 87.],
        [ 0., 25., 97.,  ..., 43., 75., 39.]])


In [217]:
def prepare_data(sort_seq_batch=False) -> Tuple[Dict[str, torch.Tensor]]:
    masked_values_train = random_mask_value(
        tokenized_train["values"],
        mask_ratio=mask_ratio,
        mask_value=mask_value,
        pad_value=pad_value,
    )
    masked_values_valid = random_mask_value(
        tokenized_valid["values"],
        mask_ratio=mask_ratio,
        mask_value=mask_value,
        pad_value=pad_value,
    )
    print(
        f"random masking at epoch {epoch:3d}, ratio of masked values in train: ",
        f"{(masked_values_train == mask_value).sum() / (masked_values_train - pad_value).count_nonzero():.4f}",
    )

    input_gene_ids_train, input_gene_ids_valid = (
        tokenized_train["genes"],
        tokenized_valid["genes"],
    )
    input_values_train, input_values_valid = masked_values_train, masked_values_valid
    
    target_values_train, target_values_valid = (
        tokenized_train["values"],
        tokenized_valid["values"],
    )

    # tensor_batch_labels_train = torch.from_numpy(train_batch_labels).long()
    # tensor_batch_labels_valid = torch.from_numpy(valid_batch_labels).long()

    tensor_celltype_labels_train = torch.from_numpy(train_celltype_labels).long()
    tensor_celltype_labels_valid = torch.from_numpy(valid_celltype_labels).long()

    # if sort_seq_batch:  # TODO: update to random pick seq source in each traning batch
    #     train_sort_ids = np.argsort(train_batch_labels)
    #     input_gene_ids_train = input_gene_ids_train[train_sort_ids]
    #     input_values_train = input_values_train[train_sort_ids]
    #     target_values_train = target_values_train[train_sort_ids]
    #     tensor_batch_labels_train = tensor_batch_labels_train[train_sort_ids]
    #     tensor_celltype_labels_train = tensor_celltype_labels_train[train_sort_ids]

    #     valid_sort_ids = np.argsort(valid_batch_labels)
    #     input_gene_ids_valid = input_gene_ids_valid[valid_sort_ids]
    #     input_values_valid = input_values_valid[valid_sort_ids]
    #     target_values_valid = target_values_valid[valid_sort_ids]
    #     tensor_batch_labels_valid = tensor_batch_labels_valid[valid_sort_ids]
    #     tensor_celltype_labels_valid = tensor_celltype_labels_valid[valid_sort_ids]

    train_data_pt = {
        "gene_ids": tokenized_train["genes"],
        "values": masked_values_train,
        # "target_values": tokenized_train["values"],
        # "batch_labels": tensor_batch_labels_train,
        "celltype_labels": tensor_celltype_labels_train,
    }
    valid_data_pt = {
        "gene_ids": tokenized_valid["genes"],
        "values": masked_values_valid,
        # "target_values": tokenized_valid["values"],
        # "batch_labels": tensor_batch_labels_valid,
        "celltype_labels": tensor_celltype_labels_valid,
    }

    return train_data_pt, valid_data_pt


In [218]:
# dataset
# 用于处理序列数据，使其可以被 PyTorch 的数据加载器 DataLoader 使用
class SeqDataset(Dataset):
    def __init__(self, data: Dict[str, torch.Tensor]):
        self.data = data

    # 获取数据集长度
    def __len__(self):
        return self.data["gene_ids"].shape[0]

    # 获取特定索引的数据å
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.data.items()}


In [219]:
# data_loader
def prepare_dataloader(
    data_pt: Dict[str, torch.Tensor],
    batch_size: int,
    shuffle: bool = False,
    intra_domain_shuffle: bool = False,
    drop_last: bool = False,
    num_workers: int = 0,
) -> DataLoader:
    if num_workers == 0:
        num_workers = min(len(os.sched_getaffinity(0)), batch_size // 2)

    dataset = SeqDataset(data_pt)

    # Initial: per_seq_batch_sample = False
    # if per_seq_batch_sample:
    #     # find the indices of samples in each seq batch
    #     subsets = []
    #     batch_labels_array = data_pt["batch_labels"].numpy()
    #     print(batch_labels_array)
    #     for batch_label in np.unique(batch_labels_array):
    #         batch_indices = np.where(batch_labels_array == batch_label)[0].tolist()
    #         subsets.append(batch_indices)
        
    #     data_loader = DataLoader(
    #         dataset=dataset,
    #         batch_sampler=SubsetsBatchSampler(
    #             subsets,
    #             batch_size,
    #             intra_subset_shuffle=intra_domain_shuffle,
    #             inter_subset_shuffle=shuffle,
    #             drop_last=drop_last,
    #         ),
    #         num_workers=num_workers,
    #         pin_memory=True,
    #     )
    #     return data_loader

    
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,
    )

    
    return data_loader

### Load the pre-trained scGPT model

In [220]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型
ntokens = len(vocab)  # size of vocabulary

model = TransformerModel(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    nlayers_cls=2, # 分类器的层数，设置为3
    n_cls=num_types if CLS else 1, # 分类器的输出类别数量。如果 CLS 为 True，则使用 num_types，否则为 1
    
    vocab=vocab,
    dropout=dropout,
    
    pad_token=pad_token, # 填充标记，用于对齐不同长度的序列
    pad_value=pad_value, # 填充值，用于对齐不同长度的序列。
    
    do_mvc=MVC, # 是否进行域自适应批归一化  // false
    do_dab=DAB, # 是否使用批次标签 //false
    use_batch_labels=INPUT_BATCH_LABELS, # false
    num_batch_labels=num_batch_types,  # use_batch_labels =  false, 这里无作用
    domain_spec_batchnorm=config.DSBN, # false
    
    input_emb_style=input_emb_style, # 输入嵌入的风格或类型 e.g, continuous
    
    n_input_bins=n_input_bins,
    
    cell_emb_style=cell_emb_style, # cls
    mvc_decoder_style=mvc_decoder_style, # do_mvc = false, 这里无作用
    
    ecs_threshold=ecs_threshold,  # set to 0.0 false
    explicit_zero_prob=explicit_zero_prob, # 是否显式地考虑零概率  // false
    
    # use_fast_transformer=fast_transformer, #是否使用快速 Transformer 实现  // True   config.fast_transformer
    use_fast_transformer=False,
    fast_transformer_backend=fast_transformer_backend, # 快速 Transformer 的后端实现  // flash attention
    
    # 前规范化：在每个子层（自注意力和前馈神经网络）之前进行规范化。这种方法可以让梯度更稳定。
    # 后规范化：在每个子层之后进行规范化。这种方法是 Transformer 最初提出时使用的方式
    pre_norm=config.pre_norm, # norm_scheme = "pre" if pre_norm else "post"  //pre_norm = false => post
)

# 加载预训练模型参数
if config.load_model is not None:
    try:
        model.load_state_dict(torch.load(model_file))
        logger.info(f"Loading all model params from {model_file}")
    except:
        # only load params that are in the model and match the size
        model_dict = model.state_dict()
        pretrained_dict = torch.load(model_file)
        pretrained_dict = {
            k: v
            for k, v in pretrained_dict.items()
            if k in model_dict and v.shape == model_dict[k].shape
        }
        for k, v in pretrained_dict.items():
            logger.info(f"Loading params {k} with shape {v.shape}")
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

# 计算冻结前需要更新的参数数量
pre_freeze_param_count = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())

# 冻结部分模型参数
for name, para in model.named_parameters():
    # print("-"*20)
    # print(f"name: {name}")
    
    # initial config.freeze = True
    # 如果参数名称包含 "encoder" 且不包含 "transformer_encoder"，则冻结该参数
    if config.freeze and "encoder" in name and "transformer_encoder" not in name: 
    # if config.freeze and "encoder" in name:  
    # if config.freeze and "encoder" in name:
        print(f"freezing weights for: {name}")
        para.requires_grad = False

# 计算冻结后需要更新的参数数量
post_freeze_param_count = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())



logger.info(f"Total Pre freeze Params {(pre_freeze_param_count )}")
logger.info(f"Total Post freeze Params {(post_freeze_param_count )}")

wandb.log(
        {
            "info/pre_freeze_param_count": pre_freeze_param_count,
            "info/post_freeze_param_count": post_freeze_param_count,
        },
)
print("\n")
print(model)

# 将模型移动到设备上并监视模型
model.to(device)
wandb.watch(model)

# #  初始化对抗性判别器（如果需要）
# if ADV:
#     discriminator = AdversarialDiscriminator(
#         d_model=embsize,
#         n_cls=num_batch_types,
#     ).to(device)



# GeneEncoder：首先对输入的基因ID进行嵌入和标准化。
# ContinuousValueEncoder：对连续值进行编码，包括线性变换、激活和标准化。
# TransformerEncoder：应用多个Transformer编码器层，对前面的编码结果进行进一步处理。



scGPT - INFO - Loading params decoder.fc.4.bias with shape torch.Size([1])
freezing weights for: encoder.embedding.weight
freezing weights for: encoder.enc_norm.weight
freezing weights for: encoder.enc_norm.bias
freezing weights for: value_encoder.embedding.weight
freezing weights for: value_encoder.enc_norm.weight
freezing weights for: value_encoder.enc_norm.bias
scGPT - INFO - Total Pre freeze Params 5144457
scGPT - INFO - Total Post freeze Params 449289


TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(36574, 128, padding_idx=36571)
    (enc_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): CategoryValueEncoder(
    (embedding): Embedding(103, 128, padding_idx=101)
    (enc_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQu

[]

In [244]:
# 损失函数
criterion_cls = nn.CrossEntropyLoss()

# 优化器
optimizer = torch.optim.Adam(
    model.parameters(), lr=lr, eps=1e-4 if config.amp else 1e-8
)

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, schedule_interval, gamma=config.schedule_ratio
)

# scaler: 使用自动混合精度的梯度缩放器 GradScaler，根据 config.amp（自动混合精度）是否启用来决定
scaler = torch.cuda.amp.GradScaler(enabled=config.amp)

In [245]:
def train(model: nn.Module, loader: DataLoader) -> None:
    """
    Train the model for one epoch.
    """
    model.train()

    # initial
    # 训练或评估循环中的部分代码
    total_loss = 0.0
    total_error = 0.0
    total_cls = 0.0
    
    start_time = time.time()

    num_batches = len(loader)
    for batch, batch_data in enumerate(loader):
        
        input_gene_ids = batch_data["gene_ids"].to(device)
        
        input_values = batch_data["values"].to(device)
        # target_values = batch_data["target_values"].to(device)
        
        # batch_labels = batch_data["batch_labels"].to(device)
        celltype_labels = batch_data["celltype_labels"].to(device)

        src_key_padding_mask = input_gene_ids.eq(vocab[pad_token])
        
        with torch.cuda.amp.autocast(enabled=config.amp):
            
            output_dict = model(
                input_gene_ids,
                input_values,
                src_key_padding_mask=src_key_padding_mask,
                # batch_labels=batch_labels if INPUT_BATCH_LABELS or config.DSBN else None,
                CLS=CLS,
                CCE=CCE,
                MVC=MVC,
                ECS=ECS,
                do_sample=do_sample_in_train,
                #generative_training=False
            )

            # masked_positions = input_values.eq(mask_value)  # the postions to predict
            
            # loss = 0.0
            
            if CLS:
                # print("output_dict\n: ",output_dict["cls_output"].argmax(1))
                # print("celltype_labels \n :",celltype_labels)
                loss_cls = criterion_cls(output_dict["cls_output"], celltype_labels)
                # loss = loss_cls
                # metrics_to_log.update({"train/cls": loss_cls.item()})

                error_rate = 1 - (
                    (output_dict["cls_output"].argmax(1) == celltype_labels)
                    .sum()
                    .item()
                ) / celltype_labels.size(0)
                
        wandb.log({"epoch": epoch,"train_loss": loss_cls.item(), "error_rate": 1-error_rate})

        # 反向传播与优化
        model.zero_grad()
        scaler.scale(loss_cls).backward()
        scaler.unscale_(optimizer)

        with warnings.catch_warnings(record=True) as w:
            warnings.filterwarnings("always")
            torch.nn.utils.clip_grad_norm_(
                model.parameters(),
                1.0,
                error_if_nonfinite=False if scaler.is_enabled() else True,
            )
            if len(w) > 0:
                logger.warning(
                    f"Found infinite gradient. This may be caused by the gradient "
                    f"scaler. The current scale is {scaler.get_scale()}. This warning "
                    "can be ignored if no longer occurs after autoscaling of the scaler."
                )
        scaler.step(optimizer)
        scaler.update()

        
        # 记录和日志
        total_loss += loss_cls.item()
        # total_cls += loss_cls.item()
        total_error += error_rate
            
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            
            cur_loss = total_loss / log_interval
            cur_cls = total_cls / log_interval
            cur_error = total_error / log_interval
            # ppl = math.exp(cur_loss)
            
            logger.info(
                f"| epoch {epoch:3d} | {batch:3d}/{num_batches:3d} batches | "
                f"lr {lr:05.4f} | ms/batch {ms_per_batch:5.2f} | "
                f"loss {cur_loss:5.2f} | "
                + (f"cls {cur_cls:5.2f} | " if CLS else "")
                + (f"err {cur_error:5.2f} | " if CLS else "")
            )
            
            total_loss = 0
            total_cls = 0
            total_error = 0
            
            start_time = time.time()

# # 定义了wandb日志中的指标
# def define_wandb_metrcis():
#     wandb.define_metric("valid/mse", summary="min", step_metric="epoch")

In [246]:
# 评估模型在验证数据集上的性能
def evaluate(model: nn.Module, loader: DataLoader, return_raw: bool = False) -> float:
    """
    Evaluate the model on the evaluation data.
    """
    model.eval()
    total_loss = 0.0
    total_error = 0.0
    total_num = 0.0
    truth_num = 0

    total = 0
    
    predictions = []
    with torch.no_grad():
        for batch_data in loader:
            input_gene_ids = batch_data["gene_ids"].to(device)
            input_values = batch_data["values"].to(device)

            # print("input_gene_ids: ",len(input_gene_ids))
            
            # target_values = batch_data["target_values"].to(device)
            # batch_labels = batch_data["batch_labels"].to(device)
            
            celltype_labels = batch_data["celltype_labels"].to(device)
            # print("celltype_labels: ",len(celltype_labels))

            src_key_padding_mask = input_gene_ids.eq(vocab[pad_token])
            with torch.cuda.amp.autocast(enabled=config.amp):
                output_dict = model(
                    input_gene_ids,
                    input_values,
                    src_key_padding_mask=src_key_padding_mask,
                    # batch_labels=batch_labels if INPUT_BATCH_LABELS or config.DSBN else None,
                    CLS=CLS,
                    CCE=CCE,
                    MVC=MVC,
                    ECS=ECS,
                    do_sample=do_sample_in_train,
                    #generative_training = False,
                )

                output_values = output_dict["cls_output"]
                loss = criterion_cls(output_values, celltype_labels)

            total_loss += loss.item() * len(input_gene_ids)
            
            truth_num = (output_values.argmax(1) == celltype_labels).sum().item()

            # print("valid_accuracy: ",accuracy / len(input_gene_ids))
            
            total_error += (1 - accuracy / len(input_gene_ids)) * len(input_gene_ids)
            total_num += len(input_gene_ids)
            
            preds = output_values.argmax(1).cpu().numpy()
            predictions.append(preds)
            
            truth_num  += truth_num

            # wandb.log({"valid_loss": loss, "valid_total_loss": total_loss, "valid_accuracy": accuracy})
            
    print("truth_num: ",truth_num)
    print("total_num: ",total_num)
    
    if return_raw:
        return np.concatenate(predictions, axis=0)

    return total_loss / total_num, total_error / total_num, total_num / len(input_gene_ids)

### Finetune scGPT with task-specific objectives

In [247]:
best_val_loss = float("inf")
best_avg_bio = 0.0
best_model = None
# define_wandb_metrcis()

wandb.watch(model, log="all")

for epoch in range(1, 5 + 1):

    print("----------------------")
    epoch_start_time = time.time()
    
    train_data_pt, valid_data_pt = prepare_data(sort_seq_batch=per_seq_batch_sample)
    
    train_loader = prepare_dataloader(
        train_data_pt,
        batch_size=batch_size,
        shuffle=False,
        intra_domain_shuffle=True,
        drop_last=False,
    )
    
    valid_loader = prepare_dataloader(
        valid_data_pt,
        batch_size=eval_batch_size,
        shuffle=False,
        intra_domain_shuffle=False,
        drop_last=False,
    )

    if config.do_train:
        train(model, loader=train_loader,)
        
    val_loss, val_err, accuracy = evaluate(model, loader=valid_loader,)

    wandb.log({"val_loss": val_loss, "val_err": val_err, "valid_accuracy": accuracy})

    # 记录和模型保存
    elapsed = time.time() - epoch_start_time
    logger.info("-" * 89)
    logger.info(
        f"| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | "
        f"valid loss/mse {val_loss:5.4f} | err {val_err:5.4f} | accuracy {accuracy:5.4f}"
    )
    logger.info("-" * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        best_model_epoch = epoch
        logger.info(f"Best model with score {best_val_loss:5.4f}")

    # 学习率调度
    scheduler.step()
    # if DAB_separate_optim:
    #     scheduler_dab.step()
    # if ADV:
    #     scheduler_D.step()
    #     scheduler_E.step()

torch.save(best_model.state_dict(), 'best_model.pth')
# wandb.finish()

----------------------


ValueError: could not convert string to float: 'auto'

In [43]:
# 在测试数据上进行模型的推理，并计算模型的性能指标，如准确率、精确率、召回率和宏观F1分数

# %% inference
def test(model: nn.Module, adata: DataLoader) -> float:
    
    all_counts = (
        adata.layers[input_layer_key].A
        if issparse(adata.layers[input_layer_key])
        else adata.layers[input_layer_key]
    )

    # label from the test data 
    celltypes_labels = adata.obs["sex_id"].tolist()  # make sure count from 0
    celltypes_labels = np.array(celltypes_labels)
    # print(celltypes_labels)

    batch_ids = adata.obs["batch_id"].tolist()
    batch_ids = np.array(batch_ids)
    # print(batch_ids)

    # 数据预处理
    tokenized_test = tokenize_and_pad_batch(
        all_counts,
        gene_ids,
        max_len=max_seq_len,
        vocab=vocab,
        pad_token=pad_token,
        pad_value=pad_value,
        append_cls=True,  # append <cls> token at the beginning
        include_zero_gene=include_zero_gene,
    )

    input_values_test = random_mask_value(
        tokenized_test["values"],
        mask_ratio=mask_ratio,
        mask_value=mask_value,
        pad_value=pad_value,
    )

    # 创建测试数据集和数据加载器
    test_data_pt = {
        "gene_ids": tokenized_test["genes"],
        "values": input_values_test,
        # "target_values": tokenized_test["values"],
        # "batch_labels": torch.from_numpy(batch_ids).long(),
        "celltype_labels": torch.from_numpy(celltypes_labels).long(),
    }

    # print(test_data_pt["celltype_labels"])

    test_loader = DataLoader(
        dataset=SeqDataset(test_data_pt),
        batch_size=eval_batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=min(len(os.sched_getaffinity(0)), eval_batch_size // 2),
        pin_memory=True,
    )


    # 模型推理和性能评估
    model.eval()
    
    predictions = evaluate(model, loader=test_loader, return_raw=True,)
    
    
    # 计算性能指标
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    accuracy = accuracy_score(celltypes_labels, predictions)
    precision = precision_score(celltypes_labels, predictions, average="macro")
    recall = recall_score(celltypes_labels, predictions, average="macro")
    macro_f1 = f1_score(celltypes_labels, predictions, average="macro")

    # 记录和返回结果
    logger.info(
        f"Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, "
        f"Macro F1: {macro_f1:.3f}"
    )

    results = {
        "test/accuracy": accuracy,
        "test/precision": precision,
        "test/recall": recall,
        "test/macro_f1": macro_f1,
    }

    return predictions, celltypes_labels, results

In [44]:
predictions, labels, results = test(best_model, adata_test)

# print(predictions)

adata_test_raw.obs["predictions"] = [p for p in predictions]

print(adata_test_raw.obs)

# print(adata_test_raw.obs["predictions"])

unique_clusters = adata_test_raw.obs['predictions'].unique()

print(unique_clusters)
# num_clusters = len(unique_clusters)
# print(f"Total {num_clusters} of the cluster labels.")

# cluster_sizes = adata_test_raw.obs['predictions'].value_counts()
# print(cluster_sizes)

# # plot
# palette_ = plt.rcParams["axes.prop_cycle"].by_key()["color"]
# palette_ = plt.rcParams["axes.prop_cycle"].by_key()["color"] + plt.rcParams["axes.prop_cycle"].by_key()["color"] + plt.rcParams["axes.prop_cycle"].by_key()["color"]
# palette_ = {c: palette_[i] for i, c in enumerate(sextypes)}

# with plt.rc_context({"figure.figsize": (6, 4), "figure.dpi": (300)}):
#     sc.pl.umap(
#         adata_test_raw,
#         color=["predictions"],
#         # color=["celltype", "predictions"],
#         palette=palette_,
#         show=False,
#     )
#     plt.savefig(save_dir / "results.png", dpi=300)

# save_dict = {
#     "predictions": predictions,
#     # "labels": labels,
#     "results": results,
#     "id_maps": id2type
# }
# with open(save_dir / "results.pkl", "wb") as f:
#     pickle.dump(save_dict, f)

# results["test/cell_umap"] = wandb.Image(
#     str(save_dir / "results.png"),
#     caption=f"predictions macro f1 {results['test/macro_f1']:.3f}",
# )

# wandb.log(results)

valid_accuracy:  0.5625
valid_accuracy:  0.4375
valid_accuracy:  0.46875
valid_accuracy:  0.53125
valid_accuracy:  0.5625
valid_accuracy:  0.5
valid_accuracy:  0.625
valid_accuracy:  0.5
valid_accuracy:  0.6875
valid_accuracy:  0.4375
valid_accuracy:  0.5625
valid_accuracy:  0.65625
valid_accuracy:  0.375
valid_accuracy:  0.625
valid_accuracy:  0.4375
valid_accuracy:  0.625
valid_accuracy:  0.625
valid_accuracy:  0.53125
valid_accuracy:  0.5
valid_accuracy:  0.625
valid_accuracy:  0.28125
valid_accuracy:  0.75
valid_accuracy:  0.53125
valid_accuracy:  0.34375
valid_accuracy:  0.5625
valid_accuracy:  0.5
valid_accuracy:  0.40625
valid_accuracy:  0.5
valid_accuracy:  0.5
valid_accuracy:  0.40625
valid_accuracy:  0.625
valid_accuracy:  0.46875
valid_accuracy:  0.4375
valid_accuracy:  0.4375
valid_accuracy:  0.5
valid_accuracy:  0.53125
valid_accuracy:  0.4375
valid_accuracy:  0.53125
valid_accuracy:  0.46875
valid_accuracy:  0.40625
valid_accuracy:  0.5
valid_accuracy:  0.5625
valid_accur